# Real-time Network Telemetry Simulation

This notebook simulates real-time network telemetry data and sends it to the SLA Prediction Platform API.

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime, timedelta
import json
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Configuration

In [ ]:
# API Configuration
API_BASE_URL = "http://localhost:8000"
PREDICT_ENDPOINT = f"{API_BASE_URL}/predict-and-store/"
ANOMALY_ENDPOINT = f"{API_BASE_URL}/anomaly/"

# Simulation parameters
SIMULATION_DURATION = 300  # seconds
UPDATE_INTERVAL = 5  # seconds
BURST_PROBABILITY = 0.1  # Probability of network burst
ANOMALY_PROBABILITY = 0.05  # Probability of anomaly

# Network nodes
NODES = ['S1', 'S2', 'S3']
NODE_PAIRS = [('S1', 'S2'), ('S1', 'S3'), ('S2', 'S3')]

## Telemetry Data Generator

In [ ]:
class NetworkTelemetryGenerator:
    """Generate realistic network telemetry data"""
    
    def __init__(self):
        self.baseline_metrics = {
            'bandwidth': 2.0,
            'throughput': 2.0,
            'congestion': 5.0,
            'packet_loss': 0.5,
            'latency': 8.0,
            'jitter': 0.5
        }
        self.time_step = 0
    
    def generate_normal_telemetry(self, source: str, target: str) -> Dict:
        """Generate normal network telemetry"""
        # Add time-based variations
        time_factor = np.sin(self.time_step * 0.1) * 0.2 + 1
        
        # Base metrics with some randomness
        telemetry = {
            'bandwidth': self.baseline_metrics['bandwidth'] * time_factor + np.random.normal(0, 0.2),
            'throughput': self.baseline_metrics['throughput'] * time_factor + np.random.normal(0, 0.3),
            'congestion': max(0, self.baseline_metrics['congestion'] + np.random.normal(0, 2)),
            'packet_loss': max(0, self.baseline_metrics['packet_loss'] + np.random.normal(0, 0.5)),
            'latency': max(1, self.baseline_metrics['latency'] + np.random.normal(0, 1.5)),
            'jitter': max(0, self.baseline_metrics['jitter'] + np.random.normal(0, 0.2)),
            'network_measure': source,
            'network_target': target,
            'routers': 'up xrv6',
            'planned_route': 'Best effort'
        }
        
        # Ensure throughput doesn't exceed bandwidth
        telemetry['throughput'] = min(telemetry['throughput'], telemetry['bandwidth'])
        
        return telemetry
    
    def generate_burst_telemetry(self, source: str, target: str) -> Dict:
        """Generate telemetry during network burst"""
        telemetry = self.generate_normal_telemetry(source, target)
        
        # Increase congestion and latency during burst
        telemetry['congestion'] *= np.random.uniform(3, 8)
        telemetry['latency'] *= np.random.uniform(2, 5)
        telemetry['packet_loss'] *= np.random.uniform(2, 10)
        telemetry['jitter'] *= np.random.uniform(2, 4)
        
        # Reduce throughput
        telemetry['throughput'] *= np.random.uniform(0.3, 0.7)
        
        return telemetry
    
    def generate_anomaly_telemetry(self, source: str, target: str) -> Dict:
        """Generate anomalous telemetry data"""
        telemetry = self.generate_normal_telemetry(source, target)
        
        # Create different types of anomalies
        anomaly_type = np.random.choice(['high_latency', 'packet_storm', 'jitter_spike', 'congestion_spike'])
        
        if anomaly_type == 'high_latency':
            telemetry['latency'] *= np.random.uniform(5, 20)
        elif anomaly_type == 'packet_storm':
            telemetry['packet_loss'] *= np.random.uniform(10, 50)
        elif anomaly_type == 'jitter_spike':
            telemetry['jitter'] *= np.random.uniform(10, 30)
        elif anomaly_type == 'congestion_spike':
            telemetry['congestion'] = np.random.uniform(80, 100)
        
        return telemetry
    
    def generate_telemetry(self, source: str, target: str) -> Dict:
        """Generate telemetry based on current conditions"""
        self.time_step += 1
        
        # Determine telemetry type
        if np.random.random() < ANOMALY_PROBABILITY:
            return self.generate_anomaly_telemetry(source, target)
        elif np.random.random() < BURST_PROBABILITY:
            return self.generate_burst_telemetry(source, target)
        else:
            return self.generate_normal_telemetry(source, target)

# Initialize generator
generator = NetworkTelemetryGenerator()

## API Client

In [ ]:
class SLAPlatformClient:
    """Client for SLA Prediction Platform API"""
    
    def __init__(self, base_url: str):
        self.base_url = base_url
        self.session = requests.Session()
    
    def predict_and_store(self, telemetry: Dict) -> Dict:
        """Send telemetry for prediction and storage"""
        try:
            response = self.session.post(
                f"{self.base_url}/predict-and-store/",
                json=telemetry,
                timeout=10
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API Error: {e}")
            return None
    
    def detect_anomaly(self, telemetry: Dict) -> Dict:
        """Send telemetry for anomaly detection"""
        try:
            response = self.session.post(
                f"{self.base_url}/anomaly/",
                json=telemetry,
                timeout=10
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Anomaly API Error: {e}")
            return None
    
    def health_check(self) -> bool:
        """Check if API is healthy"""
        try:
            response = self.session.get(f"{self.base_url}/health", timeout=5)
            return response.status_code == 200
        except:
            return False

# Initialize client
client = SLAPlatformClient(API_BASE_URL)

# Check API health
if client.health_check():
    print("✅ API is healthy and ready")
else:
    print("❌ API is not available. Please start the backend service.")

## Real-time Simulation

In [ ]:
# Data storage for visualization
simulation_data = []
predictions = []
anomalies = []

def run_simulation_step():
    """Run one step of the simulation"""
    timestamp = datetime.now()
    
    # Generate telemetry for each node pair
    for source, target in NODE_PAIRS:
        # Generate telemetry
        telemetry = generator.generate_telemetry(source, target)
        telemetry['timestamp'] = timestamp.isoformat()
        
        # Store for visualization
        simulation_data.append(telemetry.copy())
        
        # Send to API for prediction
        prediction_result = client.predict_and_store(telemetry)
        if prediction_result:
            prediction_result['timestamp'] = timestamp
            prediction_result['source'] = source
            prediction_result['target'] = target
            predictions.append(prediction_result)
            
            print(f"📊 {source}→{target}: SLA Risk {prediction_result.get('sla_violation', 'N/A')}")
        
        # Check for anomalies
        anomaly_result = client.detect_anomaly(telemetry)
        if anomaly_result and anomaly_result.get('is_anomaly'):
            anomaly_result['timestamp'] = timestamp
            anomaly_result['source'] = source
            anomaly_result['target'] = target
            anomaly_result.update(telemetry)
            anomalies.append(anomaly_result)
            
            print(f"🚨 ANOMALY {source}→{target}: {anomaly_result.get('explanation', 'Unknown')}")

# Run simulation
print(f"🚀 Starting {SIMULATION_DURATION}s simulation with {UPDATE_INTERVAL}s intervals...")
print("=" * 60)

start_time = time.time()
step = 0

while time.time() - start_time < SIMULATION_DURATION:
    step += 1
    print(f"\n⏱️  Step {step} - {datetime.now().strftime('%H:%M:%S')}")
    
    run_simulation_step()
    
    # Wait for next interval
    time.sleep(UPDATE_INTERVAL)

print("\n✅ Simulation completed!")
print(f"📈 Generated {len(simulation_data)} telemetry records")
print(f"🔮 Made {len(predictions)} predictions")
print(f"🚨 Detected {len(anomalies)} anomalies")

## Data Analysis and Visualization

In [ ]:
# Convert to DataFrames
df_telemetry = pd.DataFrame(simulation_data)
df_predictions = pd.DataFrame(predictions)
df_anomalies = pd.DataFrame(anomalies)

print(f"Telemetry records: {len(df_telemetry)}")
print(f"Predictions: {len(df_predictions)}")
print(f"Anomalies: {len(df_anomalies)}")

# Display basic statistics
if not df_telemetry.empty:
    print("\nTelemetry Statistics:")
    print(df_telemetry[['latency', 'throughput', 'packet_loss', 'jitter']].describe())

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Network Telemetry Analysis', fontsize=16)

if not df_telemetry.empty:
    # Latency over time
    axes[0, 0].plot(df_telemetry.index, df_telemetry['latency'], 'b-', alpha=0.7)
    axes[0, 0].axhline(y=10, color='r', linestyle='--', label='SLA Threshold')
    axes[0, 0].set_title('Network Latency')
    axes[0, 0].set_ylabel('Latency (ms)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Throughput vs Bandwidth
    axes[0, 1].scatter(df_telemetry['bandwidth'], df_telemetry['throughput'], alpha=0.6)
    axes[0, 1].plot([0, df_telemetry['bandwidth'].max()], [0, df_telemetry['bandwidth'].max()], 'r--', label='Ideal')
    axes[0, 1].set_title('Throughput vs Bandwidth')
    axes[0, 1].set_xlabel('Bandwidth (Mbps)')
    axes[0, 1].set_ylabel('Throughput (Mbps)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Packet Loss Distribution
    axes[1, 0].hist(df_telemetry['packet_loss'], bins=20, alpha=0.7, color='orange')
    axes[1, 0].axvline(x=5, color='r', linestyle='--', label='Threshold')
    axes[1, 0].set_title('Packet Loss Distribution')
    axes[1, 0].set_xlabel('Packet Loss (%)')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Network Congestion
    axes[1, 1].plot(df_telemetry.index, df_telemetry['congestion'], 'g-', alpha=0.7)
    axes[1, 1].axhline(y=50, color='orange', linestyle='--', label='Warning')
    axes[1, 1].axhline(y=80, color='r', linestyle='--', label='Critical')
    axes[1, 1].set_title('Network Congestion')
    axes[1, 1].set_ylabel('Congestion (%)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# SLA Violation Analysis
if not df_predictions.empty:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # SLA Violations over time
    sla_violations = [p.get('sla_violation', 0) for p in predictions]
    axes[0].plot(range(len(sla_violations)), sla_violations, 'ro-', alpha=0.7)
    axes[0].set_title('SLA Violations Over Time')
    axes[0].set_ylabel('SLA Violation (0/1)')
    axes[0].set_xlabel('Prediction Step')
    axes[0].grid(True, alpha=0.3)
    
    # SLA Violation Rate by Connection
    if 'source' in df_predictions.columns and 'target' in df_predictions.columns:
        connection_violations = df_predictions.groupby(['source', 'target'])['sla_violation'].mean()
        connection_violations.plot(kind='bar', ax=axes[1], color='coral')
        axes[1].set_title('SLA Violation Rate by Connection')
        axes[1].set_ylabel('Violation Rate')
        axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    total_predictions = len(predictions)
    total_violations = sum(sla_violations)
    violation_rate = (total_violations / total_predictions) * 100 if total_predictions > 0 else 0
    
    print(f"\n📊 SLA Analysis Summary:")
    print(f"Total Predictions: {total_predictions}")
    print(f"SLA Violations: {total_violations}")
    print(f"Violation Rate: {violation_rate:.1f}%")

In [ ]:
# Anomaly Analysis
if not df_anomalies.empty:
    print(f"\n🚨 Anomaly Analysis:")
    print(f"Total Anomalies Detected: {len(df_anomalies)}")
    
    # Anomaly types
    if 'explanation' in df_anomalies.columns:
        print("\nAnomaly Types:")
        for explanation in df_anomalies['explanation'].unique():
            count = (df_anomalies['explanation'] == explanation).sum()
            print(f"  - {explanation}: {count}")
    
    # Plot anomaly scores
    if 'anomaly_score' in df_anomalies.columns:
        plt.figure(figsize=(10, 4))
        plt.hist(df_anomalies['anomaly_score'], bins=10, alpha=0.7, color='red')
        plt.title('Anomaly Score Distribution')
        plt.xlabel('Anomaly Score')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)
        plt.show()
else:
    print("\n✅ No anomalies detected during simulation")

## Export Results

In [ ]:
# Export simulation results
timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')

if not df_telemetry.empty:
    df_telemetry.to_csv(f'simulation_telemetry_{timestamp_str}.csv', index=False)
    print(f"📁 Exported telemetry data to simulation_telemetry_{timestamp_str}.csv")

if not df_predictions.empty:
    df_predictions.to_csv(f'simulation_predictions_{timestamp_str}.csv', index=False)
    print(f"📁 Exported predictions to simulation_predictions_{timestamp_str}.csv")

if not df_anomalies.empty:
    df_anomalies.to_csv(f'simulation_anomalies_{timestamp_str}.csv', index=False)
    print(f"📁 Exported anomalies to simulation_anomalies_{timestamp_str}.csv")

# Create summary report
summary = {
    'simulation_duration': SIMULATION_DURATION,
    'update_interval': UPDATE_INTERVAL,
    'total_telemetry_records': len(simulation_data),
    'total_predictions': len(predictions),
    'total_anomalies': len(anomalies),
    'sla_violation_rate': violation_rate if 'violation_rate' in locals() else 0,
    'timestamp': datetime.now().isoformat()
}

with open(f'simulation_summary_{timestamp_str}.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"📁 Exported summary to simulation_summary_{timestamp_str}.json")
print("\n✅ Simulation complete! Check the exported files for detailed results.")